In [1]:
!pip3 install gdown
!python3 -m spacy download en
!gdown 1u2pzjXIuhqf6CR2CBI6kqxLtiynLhUEE #gender_name_csv_file
!pip3 install spacytextblob langdetect

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
     |################################| 13.9 MB 3.4 MB/s eta 0:00:01
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Downloading...
From: https://drive.google.com/uc?id=1u2pzjXIuhqf6CR2CBI6kqxLtiynLhUEE
To: /home/vkumar364/goodreads_id_name_gender_new.csv
100%|#######################################| 98.5M/98.5M [00:00<00:00, 127MB/s]


In [2]:
!pip3 install langid

In [1]:
import gzip
import json
import os
import pandas as pd
import langid

def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)        
    return data

In [4]:
!gdown 1NpFsDQKBj_lrTzSASfyKbmkSykzN88wE

Downloading...
From: https://drive.google.com/uc?id=1NpFsDQKBj_lrTzSASfyKbmkSykzN88wE
To: /home/vkumar364/goodreads_reviews_romance.json.gz
100%|#######################################| 1.30G/1.30G [00:11<00:00, 112MB/s]


In [2]:
def download_and_join_data(ggid,csvname,gendered_df):
#     os.system('gdown '+ ggid)
    data = load_data(csvname)
    df = pd.DataFrame(data)
    final = pd.merge(df, gendered_data, on='user_id', how='left')
    return final[['user_id', 'rating', 'review_text', 'n_votes','n_comments','gender']]

In [3]:
gendered_data = pd.read_csv('goodreads_id_name_gender_new.csv')
gendered_data = gendered_data.drop(columns=['Unnamed: 0','name_x','age','location','friends_count','groups_count'])
gendered_data = gendered_data.rename(columns={'uuid':'user_id'})
gendered_data.head()

,id,username,joined,reviews_count,first_name,user_id,gender
0,50002191,sozerseda,12/2015,184.0,seda,aad6531213b6711ec6bfa5cd3ecac285,female
1,58298653,NaN,08/2016,24.0,olivia,30a37d704d9c28681606a3fbe80ed58f,female
2,36210433,NaN,10/2014,379.0,naomi,f856f6ef2eb4ad702af863175f281f92,female
3,6949799,azima,11/2011,230.0,azima,32d2876b7eb3db5a1ee87f29c01dce4c,female
4,21443818,NaN,NaN,NaN,nicholas,2ace7a7c6edcdd8ac788fe4fd552b7ca,male


In [4]:
selected = download_and_join_data('1THnnmE4XSCvMkGOsqapQr2cJI5amKA6X','goodreads_reviews_romance.json.gz', gendered_data)

In [5]:
selected['gender'].value_counts()

female    2615311
UNK        331642
male       298117
Name: gender, dtype: int64

In [10]:
selected_final = selected[selected['gender']!='UNK']

In [11]:
selected_final['gender'].value_counts()

female    2615311
male       298117
Name: gender, dtype: int64

In [12]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
!pip install contractions

[nltk_data] Downloading package stopwords to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /nethome/vkumar364/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [14]:
import contractions 
import re
from nltk.corpus import stopwords
import numpy as np

In [15]:
expand_contractions = True
remove_stopwords = False
format_text = True
tokenize = False
lemmatize = False

def clean_text(text):
    '''Text Preprocessing '''
    
    # Convert words to lower case
    text = text.lower()
    split_text = text.split()
    
    # Expand contractions
    if expand_contractions:
        expanded_words = []   
        for word in split_text:
            expanded_words.append(contractions.fix(word))
        text = " ".join(expanded_words)

    # Format words and remove unwanted characters
    if format_text:
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text) 
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)
    
    # remove stopwords
    if remove_stopwords:
        split_text = text.split()
        stops = set(stopwords.words("english"))
        word = [w for w in split_text if not w in stops]
        text = " ".join(word)

    # Tokenize each word
    if tokenize:
        text =  nltk.WordPunctTokenizer().tokenize(text)
    
    # Lemmatize each token
    if lemmatize:
        lemm = nltk.stem.WordNetLemmatizer()
        text = list(map(lambda word:list(map(lemm.lemmatize, word)), text))
    
    return text

In [16]:
selected['review_length'] = selected['review_text'].apply(lambda x: len(x.split(' ')))

In [17]:
def get_subsampled_gendered(selected_final,lower,upper):
    selected_final_male = selected_final[selected_final['gender']=='male']
    selected_final_female = selected_final[selected_final['gender']=='female']
    selected_final_male_rev = selected_final_male[selected_final_male['review_length']>lower]
    selected_final_male_rev = selected_final_male_rev[selected_final_male_rev['review_length']<=upper]
    selected_final_female_rev = selected_final_female[selected_final_female['review_length']>lower]
    selected_final_female_rev = selected_final_female_rev[selected_final_female_rev['review_length']<=upper]
    #subsampling here
    min_select = min(list(selected_final_female_rev['rating'].value_counts()) + list(selected_final_male_rev['rating'].value_counts()))
    #sampling down the total data - for males
    x = selected_final_male_rev[selected_final_male_rev['rating']==1].sample(min_select)
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==2].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==3].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==4].sample(min_select))
    x = x.append(selected_final_male_rev[selected_final_male_rev['rating']==5].sample(min_select))
    #sampling down the total data - for females
    y = selected_final_female_rev[selected_final_female_rev['rating']==1].sample(min_select)
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==2].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==3].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==4].sample(min_select))
    y = y.append(selected_final_female_rev[selected_final_female_rev['rating']==5].sample(min_select))
    return x,y
    

In [18]:
male_rev,female_rev = get_subsampled_gendered(selected,5,500)

In [19]:
def clean_en_sentiment(df):
    df['review_text_cleaned'] = df['review_text'].apply(clean_text)
    df['review_lang'] = df['review_text'].apply(lambda x: langid.classify(x)[0])
    df = df[df['review_lang'] == 'en']
    df['sentiment_score'] = df['review_text'].apply(lambda x: nlp(x)._.polarity)
    return df 

In [20]:
male_rev_cleaned = clean_en_sentiment(male_rev)
female_rev_cleaned = clean_en_sentiment(female_rev)

/tmp/ipykernel_29485/1754966298.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_score'] = df['review_text'].apply(lambda x: nlp(x)._.polarity)


In [21]:
female_rev_cleaned = female_rev_cleaned.reset_index()
male_rev_cleaned = male_rev_cleaned.reset_index()

In [22]:
male_rev_cleaned = male_rev_cleaned[['rating','n_votes', 'n_comments', 'gender','review_length','sentiment_score']]
female_rev_cleaned = female_rev_cleaned[['rating','n_votes', 'n_comments', 'gender','review_length','sentiment_score']]

In [23]:
male_rev_cleaned

,rating,n_votes,n_comments,gender,review_length,sentiment_score
0,1,0,0,male,236,0.065179
1,1,2,0,male,450,0.041031
2,1,0,0,male,15,-1.000000
3,1,0,0,male,31,-0.235417
4,1,1,0,male,174,0.137564
...,...,...,...,...,...,...
33729,5,0,0,male,83,0.287500
33730,5,0,1,male,21,0.000000
33731,5,0,0,male,302,0.242222
33732,5,0,0,male,161,0.147619


In [24]:
male_rev_cleaned['gender'] = 0
female_rev_cleaned['gender'] = 1

In [25]:
combined_data = male_rev_cleaned.append(female_rev_cleaned)

In [26]:
combined_data

,rating,n_votes,n_comments,gender,review_length,sentiment_score
0,1,0,0,0,236,0.065179
1,1,2,0,0,450,0.041031
2,1,0,0,0,15,-1.000000
3,1,0,0,0,31,-0.235417
4,1,1,0,0,174,0.137564
...,...,...,...,...,...,...
34676,5,0,0,1,119,0.294697
34677,5,0,0,1,49,0.282143
34678,5,0,0,1,170,0.214132
34679,5,0,0,1,401,0.249302


In [27]:
final_g = combined_data[['n_votes', 'n_comments', 'gender', 'review_length','sentiment_score']]
final_w = combined_data[['n_votes', 'n_comments', 'review_length','sentiment_score']]
pred = combined_data['rating']

In [28]:
final_g.head()

,n_votes,n_comments,gender,review_length,sentiment_score
0,0,0,0,236,0.065179
1,2,0,0,450,0.041031
2,0,0,0,15,-1.000000
3,0,0,0,31,-0.235417
4,1,0,0,174,0.137564


In [29]:
import statsmodels.api as sm
model_g = sm.OLS(pred, sm.add_constant(final_g)).fit()
model_w = sm.OLS(pred, sm.add_constant(final_w)).fit()

/nethome/vkumar364/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [30]:
model_g.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     4141.
Date:                Thu, 28 Apr 2022   Prob (F-statistic):               0.00
Time:                        16:13:00   Log-Likelihood:            -1.1200e+05
No. Observations:               68415   AIC:                         2.240e+05
Df Residuals:                   68409   BIC:                         2.241e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.4293      0.009    265.966      0.000       2.411       2.447
n_votes             0.0031      0.001      2.959      0.003       0.001       0.005
n_comments         -0.0240      0.002    -10.807      0.000      -0.028      -0.020
gender             -0.0182      0.010     -1.913      0.056      -0.037       0.000
review_length       0.0013   4.13e-05     31.184      0.000       0.001       0.001
sentiment_score     2.8601      0.020    141.003      0.000       2.820       2.900
==============================================================================
Omnibus:                     2957.378   Durbin-Watson:                   0.461
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1235.516
Skew:                           0.001   Prob(JB):                    5.14e-269
Kurtosis:                       2.342   Cond. No.                         719.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
sm.stats.anova_lm(model_w, model_g)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,68410.0,105835.978081,0.0,NaN,NaN,NaN
1,68409.0,105830.315898,1.0,5.662183,3.66005,0.055736


In [32]:
import gc
gc.collect()

432